In [3]:

from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [5]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

ModuleNotFoundError: No module named 'google.colab'

In [6]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [8]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [9]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [11]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [13]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [14]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [15]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [17]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 4.75504785e-04  4.61823773e-03 -5.14209270e-03 ...  7.87335541e-03
    3.17024160e-03  6.81129610e-03]
  [ 7.00788898e-03  1.27234031e-03 -7.73851946e-03 ...  7.52714509e-03
    5.38814859e-03  4.84442152e-03]
  [ 9.04280692e-03  3.68086621e-03  1.77792297e-03 ...  1.12682935e-02
   -1.19009055e-05  4.37266799e-03]
  ...
  [ 4.03240556e-03  3.96097498e-03  1.29594486e-02 ... -2.66650715e-03
    8.79375264e-04  4.47087642e-03]
  [ 4.73749638e-03  7.04836007e-03  1.20583605e-02 ...  1.22759771e-03
    2.18803645e-03  8.74364376e-03]
  [ 1.28287077e-02  7.55959563e-03  1.43199805e-02 ... -5.37481625e-04
   -3.92527971e-03  6.14569429e-03]]

 [[ 5.85139496e-04 -1.94345647e-03 -2.19426001e-03 ... -3.47437407e-03
    1.95373083e-03  1.03279261e-03]
  [-1.64454465e-03 -4.93844505e-03 -1.73081085e-03 ...  4.01507551e-03
    3.01364949e-03 -6.58861420e-04]
  [ 2.50769313e-03 -7.18995370e-03 -3.35725490e-03 ...  4.28408664e-03
    4.69705230e-03  4.30947915e-03]
  ...
  [ 1.486

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 4.75504785e-04  4.61823773e-03 -5.14209270e-03 ...  7.87335541e-03
   3.17024160e-03  6.81129610e-03]
 [ 7.00788898e-03  1.27234031e-03 -7.73851946e-03 ...  7.52714509e-03
   5.38814859e-03  4.84442152e-03]
 [ 9.04280692e-03  3.68086621e-03  1.77792297e-03 ...  1.12682935e-02
  -1.19009055e-05  4.37266799e-03]
 ...
 [ 4.03240556e-03  3.96097498e-03  1.29594486e-02 ... -2.66650715e-03
   8.79375264e-04  4.47087642e-03]
 [ 4.73749638e-03  7.04836007e-03  1.20583605e-02 ...  1.22759771e-03
   2.18803645e-03  8.74364376e-03]
 [ 1.28287077e-02  7.55959563e-03  1.43199805e-02 ... -5.37481625e-04
  -3.92527971e-03  6.14569429e-03]], shape=(100, 65), dtype=float32)


In [20]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 4.7550478e-04  4.6182377e-03 -5.1420927e-03 -2.7075331e-03
  1.9919076e-03 -2.4848888e-03 -3.5748602e-04  6.0388539e-04
  6.6918861e-03  3.4856100e-03 -1.3167015e-03  3.9229980e-03
 -4.7985470e-04  2.2156076e-03 -2.9186388e-03 -7.7367498e-04
 -5.8497889e-03 -7.7232893e-04  1.4294786e-03 -1.1423768e-03
 -1.3459530e-03 -1.0931921e-03  2.8382558e-03  1.6998771e-03
 -1.4126454e-03 -5.2349479e-03  6.7779007e-03 -4.2938799e-04
  2.0984532e-03  5.0868997e-03  2.5977381e-05  3.6634705e-03
  3.0609127e-04  2.8621682e-03 -9.2503330e-04 -3.0450424e-04
  3.7992690e-03 -3.7778311e-03 -1.6563362e-03 -5.6621921e-04
 -1.6072134e-03 -8.9809299e-05  1.7469371e-03  4.1930904e-03
  3.0617772e-03 -4.8131449e-04  6.2694484e-03  3.0486328e-03
  4.0703029e-03  6.8066077e-04 -5.1814306e-04  1.3293519e-03
 -4.4815894e-03 -2.0634539e-03 -1.6131334e-04  4.4770818e-04
 -2.1122049e-03  2.4928508e-04  5.4204669e-03  2.1751286e-03
  1.6434235e-04 -1.2351661e-03  7.8733554e-03  3.1702416e-03
  6.811296

In [21]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"!.cAdG-fjhOopOYX\nj FY3AN!:nvE.SmG;oo!-XZwHvoZntc.h'\nHTlkRj!Q-EBu,s,dlmK-$BTqd\n3OmF,HcC,3ZEVFZHuryJKi"

In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
history = model.fit(data, epochs=2, callbacks=[checkpoint_callback])

Epoch 1/2
172/172 [==============================] - 888s 5s/step - loss: 3.0390
Epoch 2/2
172/172 [==============================] - 797s 5s/step - loss: 1.9711


In [26]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [27]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ./training_checkpoints/ckpt_10

In [30]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [31]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: hello
hellow wene aricubds!
and to was freed for any bear privine,
This father, art should ouths?

DICHARD:
Now; new,,
The, thouer tersios fird bear.

ISGBELE:
A har'f ars lusbandal Butingen,
Heave ang upper hath not thy king, wirl coneract
Stan my seed that liked Mysuck; I hapime,
Bidse fragherand,
The peallat of awligaly levines so-elvest your weremone
That to the pawn, and agaid. Beisnoy, go dembly of soor
Lelisens Hantickle child, stary is, though thut feart,
That hajour gr of Gich be go him, will,
I meet the forsh
Ot comururou, for moriom orer's notle;
Ande, and I dmant you wran Mardu: on ene:
By
Ko wabwers it feOr.

DULUEN EANGARE:
In datuif
If that pact legher, ach thrinal!
I an the verer's leevimt.
To hone in your gorditionce Ond time if ret fiththourd.

KING RICHARD II:
Yet, you hir for thee
